In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import csv
import pickle

In [2]:
# 读取数据
fresh_data = pd.read_excel("C:/Users/Administrator/Desktop/帮导师干的活/多阶段抽样/1HZH/1.xlsx", encoding="utf-8")
fresh_data.head()

,月份,编码,数量,金额,说明,人员,户,jie,ju
0,1,210111,0.0,7580.0,0,2,008H00401,万寿路街道办事处,朱各庄社区居委会普查区第007普
1,1,210111,0.0,4680.0,0,1,008H00401,万寿路街道办事处,朱各庄社区居委会普查区第007普
2,1,240111,0.0,3630.0,0,3,008H00401,万寿路街道办事处,朱各庄社区居委会普查区第007普
3,1,240111,0.0,8160.0,0,4,008H00401,万寿路街道办事处,朱各庄社区居委会普查区第007普
4,1,311012,5.0,21.6,0,99,008H00401,万寿路街道办事处,朱各庄社区居委会普查区第007普


In [7]:
len(fresh_data["ju"].unique()),len(fresh_data["jie"].unique())

(70, 26)

In [9]:
fresh_data.drop(["数量","说明","人员"],axis=1,inplace=True)

In [10]:
def code_func(x):
    str_x = str(x)
    if str_x[0] == "6":
        return 6
    else:
        return int(str(x)[:2])

fresh_data["编码"] = fresh_data["编码"].apply(code_func)
fresh_data.sort_values("编码", ascending=True, inplace=True)
fresh_data.head()

,月份,编码,金额,户,jie,ju
4781,2,6,0.0,008H03801,万寿路街道办事处,太平路44号社区居委会普查区第00
48336,3,6,3120.0,013H13601,八里庄街道办事处,定慧西里第一社区居委会普查区第
119155,2,6,0.0,002H07601,海淀街道办事处,万泉新新家园社区居委会普查区第
119268,3,6,2760.0,002H07601,海淀街道办事处,万泉新新家园社区居委会普查区第
48189,2,6,0.0,013H13601,八里庄街道办事处,定慧西里第一社区居委会普查区第


In [11]:
data = fresh_data.groupby(by=["户","jie","ju","月份","编码"], as_index=False).sum()
data.head()

,户,jie,ju,月份,编码,金额
0,001H00801,温泉镇,温泉村委会第001调查小区,1,6,0.00
1,001H00801,温泉镇,温泉村委会第001调查小区,1,21,1600.00
2,001H00801,温泉镇,温泉村委会第001调查小区,1,25,480.00
3,001H00801,温泉镇,温泉村委会第001调查小区,1,31,354.65
4,001H00801,温泉镇,温泉村委会第001调查小区,1,32,23.00


In [12]:
bouns_type = ["其他记账项目","工资性收入","非农业生产经营收入","财产性收入","转移性收入",
                 "非收入所得","借贷性所得","食品烟酒","衣着","居住","生活用品及服务","交通通信","教育文化娱乐",
                 "医疗保健","其他用品及服务","从单位或雇主得到的实物和服务","从政府和组织得到的实物和服务","非农业生产经营费用",
                 "财产性支出","转移性支出","商业保险支出","购置资产及非经常性转移支出","借贷性支出","能源消费和购买"]
bouns_index = [6]+list(range(21,27))+list(range(31,39))+[41,42]+list(range(51,57))+[78]
bouns_code_df = pd.DataFrame(np.array(bouns_type).reshape(len(bouns_type),1),columns=["收支种类"],index=bouns_index)

In [13]:
csvfile = open("C:/Users/Administrator/Desktop/summary1.csv", "w+", encoding="utf-8",newline="")
writer = csv.writer(csvfile)
writer.writerow(["户","街道","居委会","月份"]+bouns_type)

191

In [14]:
len(fresh_data["户"].unique())

688

In [15]:
x = 0
hu_id = data["户"].unique()
n = len(hu_id)
for hu in hu_id:
    every_hu = data.loc[data["户"]==hu, :]
    jie = every_hu["jie"].unique()
    for j in jie:
        every_jie = every_hu.loc[every_hu["jie"]==j, :]
        qu = every_jie["ju"].unique()
        for q in qu:
            every_qu = every_jie.loc[every_jie["ju"]==q, :]
            month = every_qu["月份"].unique()
            for m in month:
                every_hu_month = every_qu.loc[every_qu["月份"]==m, :]
                out_put_data = pd.merge(every_hu_month, bouns_code_df, left_on="编码", right_index=True, how="outer")
                out_put_data.fillna({"金额":0}, inplace=True)
                out_put_data.fillna(method="ffill", inplace=True)
                out_put_data.sort_values(by=["户","月份","编码"],inplace=True)
                writer_data = list(every_hu_month.iloc[0, :4]) + list(out_put_data["金额"].values)
                writer.writerow(writer_data)
              
    x+=1
    print(x)
            
csvfile.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
xx = pd.read_excel("C:/Users/Administrator/Desktop/帮导师干的活/多阶段抽样/1HZH/1.xls", encoding="utf-8")

In [3]:
len(xx.drop_duplicates(["户","街道","居委会"]))

746

In [26]:
len(fresh_data.drop_duplicates(["户","jie","ju"]))

746

In [10]:
xx.loc[xx["月份"]==3, "户":"居委会"].drop_duplicates()

,户,街道,居委会
2,001H00801,温泉镇,温泉村委会第001调查小区
5,001H01702,温泉镇,温泉村委会第001调查小区
8,001H02101,温泉镇,温泉村委会第001调查小区
11,001H02701,温泉镇,温泉村委会第001调查小区
14,001H03601,温泉镇,温泉村委会第001调查小区
17,001H04301,北下关街道办事处,大柳树北社区居委会普查区第001
20,001H04501,温泉镇,温泉村委会第001调查小区
23,001H04901,温泉镇,温泉村委会第001调查小区
26,001H05001,北下关街道办事处,大柳树北社区居委会普查区第001
29,001H05201,北下关街道办事处,大柳树北社区居委会普查区第001


In [11]:
xx.loc[:,:"居委会"].drop_duplicates()

,户,街道,居委会
0,001H00801,温泉镇,温泉村委会第001调查小区
3,001H01702,温泉镇,温泉村委会第001调查小区
6,001H02101,温泉镇,温泉村委会第001调查小区
9,001H02701,温泉镇,温泉村委会第001调查小区
12,001H03601,温泉镇,温泉村委会第001调查小区
15,001H04301,北下关街道办事处,大柳树北社区居委会普查区第001
18,001H04501,温泉镇,温泉村委会第001调查小区
21,001H04901,温泉镇,温泉村委会第001调查小区
24,001H05001,北下关街道办事处,大柳树北社区居委会普查区第001
27,001H05201,北下关街道办事处,大柳树北社区居委会普查区第001
